⚠️ Need to have .env file in the project root with OPENAI_API_KEY={your_openai_key}

In [ ]:
!pip install langchain openai python-dotenv

In [7]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

with open('data/jenkins-pipeline.txt', 'r') as file:
    contents = file.read()

chat = ChatOpenAI(model = 'gpt-3.5-turbo-16k-0613', temperature = 0.7, openai_api_key=openai_api_key)
system_message_prompt = SystemMessagePromptTemplate.from_template_file('data/system_prompt.txt', [])
human_message_prompt = HumanMessagePromptTemplate.from_template_file('data/human_prompt.txt', input_variables = ['pipeline']).format_messages(pipeline = contents)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt,
     MessagesPlaceholder(variable_name='history')] +
     human_message_prompt +
     [HumanMessagePromptTemplate.from_template("{input}")]
)

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=chat_prompt, llm=chat)

response = conversation.predict(input = '')

print(response)

1. Combine the 'Build' and 'Test' stages into a single stage to reduce the number of steps and optimize pipeline execution.
2. Move the 'Static Code Analysis' stage before the 'Test' stage to catch any potential issues earlier in the pipeline.
3. Consider parallelizing the 'Integration Test' and 'Code Coverage' stages to run them concurrently, if possible, to reduce pipeline execution time.
4. Move the 'Package Artifacts' stage before the 'Archive Artifacts' stage to ensure that the artifacts are packaged before archiving them.
5. Add a condition in the 'Archive Artifacts' stage to only archive artifacts if the build is successful, to avoid archiving failed builds.
6. Consider using a separate stage for deploying to GCP, instead of embedding it within the pipeline, for better separation of concerns.
7. Use a separate step for activating the GCP service account instead of doing it in the same step as setting the project, for better readability.
8. Consider using a Jenkins plugin for dep

In [11]:
user_prompt_template = """Please apply only the items I list below and provide the updated pipeline. The output should contain only the new pipeline without any supportive text above or below the updated pipeline. The output should be easily parseable programmatically.
Items I selected: {items}"""
user_prompt = HumanMessagePromptTemplate.from_template(user_prompt_template).format(items = '1,2,3,5,8,10').content
print(conversation.predict(input = user_prompt))

1. Combine the 'Build' and 'Test' stages into a single stage to reduce the number of steps and optimize pipeline execution.
2. Move the 'Static Code Analysis' stage before the 'Code Quality' stage to catch any potential issues earlier in the pipeline.
3. Consider parallelizing the 'Integration Test' and 'Code Coverage' stages to run them concurrently, if possible, to reduce pipeline execution time.
5. Add a condition in the 'Archive Artifacts' stage to only archive artifacts if the build is successful, to avoid archiving failed builds.
8. Consider using a Jenkins plugin for deploying to GCP instead of using command line commands, if available, for easier maintenance and better integration with the Jenkins ecosystem.
10. Ensure that the necessary plugins and dependencies are installed in the Jenkins environment to support the commands and steps used in the pipeline.

Updated Pipeline:
pipeline {
    agent any

    environment {
        GCP_PROJECT = 'test-project' // Replace with your G